This notebook is used to initialize the directory structure for the materials directory using an example YAML (or json) file of a Materials Project document.

In [41]:
import yaml
import os
import shutil
import json

In [42]:
desired = set()

example = """## Example response in JSON

```json
%s
```

"""

def make_dir(d, rootdir):
    for k, v in d.items():
        if k == "_id":
            # Ignore object id key, which is useless.
            continue
        subpath = os.path.join(rootdir, k)
        try:
            os.makedirs(subpath)
        except:
            pass
        fname = os.path.join(subpath, "README.md")
        desired.add(subpath)
        desired.add(fname)
        if not os.path.exists(fname):
            with open(fname, "w") as f:
                if v is not None:
                    data = example % json.dumps(v, indent=4)
                else:
                    data = ""
                f.write(data)
        else:
            with open(fname) as f:
                data = f.read().strip()
            data = data.split("## Example response in JSON")[0]
            if "## Example response in JSON" not in data:
                if v is not None:
                    data += "\n\n"
                    data += example % json.dumps(v, indent=4)
                    with open(fname, "w") as f:
                        print(f"Adding example to {k}")
                        f.write(data)

        if isinstance(v, dict) and "formula" not in k:
            # Ignore formulas which has material specific keys.
            make_dir(v, rootdir=subpath)

In [43]:
for fname in [fname for fname in listdir('.') if fname.endswith('.yaml')]:
    with open(fname) as f:
        doc = yaml.load(f)
    make_dir(doc, rootdir="../materials")

Adding example to anonymous_formula
Adding example to band_gap
Adding example to search_gap
Adding example to band_gap
Adding example to is_direct
Adding example to band_structure
Adding example to GGA
Adding example to task_id
Adding example to blessed_tasks
Adding example to GGA NSCF Line
Adding example to GGA NSCF Uniform
Adding example to GGA Static
Adding example to GGA Static Dielectric
Adding example to GGA Structure Optimization
Adding example to bond_valence
Adding example to method
Adding example to possible_species
Adding example to possible_valences
Adding example to chemsys
Adding example to cif
Adding example to cifs
Adding example to computed
Adding example to conventional_standard
Adding example to primitive
Adding example to refined
Adding example to created_at
Adding example to delta_volume
Adding example to density
Adding example to diel
Adding example to e_electronic
Adding example to e_total
Adding example to n
Adding example to poly_electronic
Adding example to po

Adding example to gamma
Adding example to matrix
Adding example to volume
Adding example to sites
Adding example to input
Adding example to crystal
Adding example to @class
Adding example to @module
Adding example to lattice
Adding example to a
Adding example to alpha
Adding example to b
Adding example to beta
Adding example to c
Adding example to gamma
Adding example to matrix
Adding example to volume
Adding example to sites
Adding example to incar
Adding example to @class
Adding example to @module
Adding example to ALGO
Adding example to EDIFF
Adding example to ENCUT
Adding example to IBRION
Adding example to ICHARG
Adding example to ISIF
Adding example to ISMEAR
Adding example to ISPIN
Adding example to LREAL
Adding example to LWAVE
Adding example to MAGMOM
Adding example to NELM
Adding example to NELMIN
Adding example to NPAR
Adding example to NSW
Adding example to PREC
Adding example to SIGMA
Adding example to SYSTEM
Adding example to kpoints
Adding example to @class
Adding exampl

In [44]:
desired = {d for d in desired if "../materials/sbxn" not in d or "../materials/sbxd" not in d}

In [58]:
paths_to_kill = set(glob.glob("../materials/**", recursive=True))
paths_to_kill -= desired
paths_to_kill -= {"../materials/"}
paths_to_kill =  {d for d in desired if "../materials/sbxn" in d or "../materials/sbxd" in d}

In [59]:
import glob
from tqdm import tqdm_notebook as tqdm

for path in tqdm(paths_to_kill):
    try:
        os.remove(path)
    except OSError:
        try:
            os.removedirs(path)
        except OSError:
            pass